In [1]:
!pip install flask
!pip install line-bot-sdk

    1. 整體功能描述

    2. 應用伺服器主結構樣板
         用來定義使用者傳送消息時，應該傳到哪些方法上
           比如收到文字消息時，轉送到文字專屬處理方法

    3. 消息專屬方法定義
         當收到文字消息，從資料夾內提取消息，並進行回傳。

    4. 啟動應用伺服器

# Application 主架構

In [2]:
# 引用 Web Server 套件
from flask import Flask, request, abort

# 從 linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (LineBotApi, WebhookHandler)

# 引用無效簽章錯誤
from linebot.exceptions import InvalidSignatureError

# 載入 json 處理套件
import json

# 載入基礎設定檔
secretFileContentJson = json.load(open("./line_secret_key", 'r', encoding='utf8'))

# 設定 Server 啟用細節
app = Flask(__name__, static_url_path = "/material" , static_folder = "./material/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動 server 對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

# 消息判斷器
    讀取指定的 json 檔案後，把 json 解析成不同格式的 SendMessage

    讀取檔案，把內容轉換成 jsonn 將 json 轉換成消息放回 array 中，並把 array 傳出。

In [3]:
# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage, 
    TextSendMessage, 
    ImageSendMessage, 
    StickerSendMessage,
    AudioSendMessage,
    LocationSendMessage, 
    FlexSendMessage, 
    VideoSendMessage
)

from linebot.models.template import (
    ButtonsTemplate, CarouselTemplate, ConfirmTemplate, ImageCarouselTemplate,
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    # 開啟檔案，轉成 json
    with open(fileName, 'r', encoding='utf8') as f:
        jsonArray = json.load(f)
    
    # 解析 json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'video':
            returnArray.append(VideoSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

## handler 處理關注消息
    用戶關注時，讀取 素材 -> 關注 -> reply.json

    將其轉換成可寄發的消息，傳回給 Line

In [4]:
# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    # 讀取並轉換
    replyJsonPath = "material/follow/reply.json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )
    
    # 從 follow 資料夾內，取出圖文選單 id，並告知 line，綁定用戶
    linkRichMenuId = open("material/follow/rich_menu_id", 'r').read()
    line_bot_api.link_rich_menu_to_user(event.source.user_id, linkRichMenuId)
    

## handler 處理文字消息
    收到用戶回應的文字消息，
    按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的 reply.json

    轉譯 json 後，將消息回傳給用戶

In [5]:
# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):

    # 讀取本地檔案，並轉譯成消息
    result_message_array = []
    replyJsonPath = f"material/{ event.message.text }/reply.json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

    # 發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )


## handler 處理 Postback Event
    載入功能選單與啟動特殊功能

    解析 postback 的 data，並按照 data 欄位判斷處理

    現有三個欄位
    menu, folder, tag

    若 folder 欄位有值，則
        讀取其 reply.json，轉譯成消息，並發送

    若 menu 欄位有值，則
        讀取其 rich_menu_id，並取得用戶 id，將用戶與選單綁定
        讀取其 reply.json，轉譯成消息，並發送

In [6]:
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 

@handler.add(PostbackEvent)
def process_postback_event(event):
    
    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'folder' in query_string_dict:
    
        result_message_array = []

        replyJsonPath = f"material/{ query_string_dict.get('folder')[0] }/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'menu' in query_string_dict:

        linkRichMenuId = open(f"material/{ query_string_dict.get('menu')[0] }/rich_menu_id", 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        replyJsonPath = f"material/{ query_string_dict.get('menu')[0] }/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )

# Application 運行（開發版）

In [ ]:
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Apr/2020 16:24:37] "POST / HTTP/1.1" 400 -
127.0.0.1 - - [19/Apr/2020 16:25:06] "POST / HTTP/1.1" 400 -
127.0.0.1 - - [19/Apr/2020 16:25:07] "POST / HTTP/1.1" 400 -
127.0.0.1 - - [19/Apr/2020 16:26:09] "POST / HTTP/1.1" 400 -
127.0.0.1 - - [19/Apr/2020 16:26:17] "POST / HTTP/1.1" 400 -
127.0.0.1 - - [19/Apr/2020 16:26:27] "POST / HTTP/1.1" 400 -
127.0.0.1 - - [19/Apr/2020 16:28:19] "POST / HTTP/1.1" 200 -
[2020-04-19 16:28:20,454] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc

{'menu': ['menu_up']}


[2020-04-19 16:33:26,535] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-2-7f1e5cc111

{'menu': ['index']}


[2020-04-19 16:33:28,573] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\his62\Anaconda3\envs\nlp\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-2-7f1e5cc111

# Application 運行（heroku版）

In [ ]:
# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])